<a href="https://colab.research.google.com/github/MrPhipps/Colabs/blob/main/RAVEv2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title README
!yes | pip install --quiet pyfiglet

import pyfiglet

resulta = pyfiglet.figlet_format(f"RAVE", font = "isometric1", )
resultb = pyfiglet.figlet_format(f"v2", font = "isometric2", )
print(resulta)
# print(resultb)
print("\n")
print("\n")
print("RAVE: Realtime Audio Variational autoEncoder | Authors: Antoine Caillon, Acids-IRCAM | https://github.com/acids-ircam/RAVE | https://arxiv.org/abs/2111.05011")
print("Custom collab implementation by Moisés Horta | http://twitter.com/hexorcismos | http://github.com/moiseshorta")
print("\n")
print("Last Modified: 2.12.2024")

      ___           ___           ___           ___     
     /\  \         /\  \         /\__\         /\  \    
    /::\  \       /::\  \       /:/  /        /::\  \   
   /:/\:\  \     /:/\:\  \     /:/  /        /:/\:\  \  
  /::\~\:\  \   /::\~\:\  \   /:/__/  ___   /::\~\:\  \ 
 /:/\:\ \:\__\ /:/\:\ \:\__\  |:|  | /\__\ /:/\:\ \:\__\
 \/_|::\/:/  / \/__\:\/:/  /  |:|  |/:/  / \:\~\:\ \/__/
    |:|::/  /       \::/  /   |:|__/:/  /   \:\ \:\__\  
    |:|\/__/        /:/  /     \::::/__/     \:\ \/__/  
    |:|  |         /:/  /       ~~~~          \:\__\    
     \|__|         \/__/                       \/__/    





RAVE: Realtime Audio Variational autoEncoder | Authors: Antoine Caillon, Acids-IRCAM | https://github.com/acids-ircam/RAVE | https://arxiv.org/abs/2111.05011
Custom collab implementation by Moisés Horta | http://twitter.com/hexorcismos | http://github.com/moiseshorta


Last Modified: 2.12.2024


In [ ]:
#@title Install Conda, RAVE and ffmpeg


!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /content/miniconda
!/content/miniconda/bin/pip install --quiet acids-rave
!/content/miniconda/bin/pip install --quiet --upgrade ipython ipykernel
!/content/miniconda/bin/conda install ffmpeg

In [ ]:
#@title Tensorboard for monitoring training

logs_path = "/path/to/rave/folder"#@param {type:"string"}

%cd $logs_path
%load_ext tensorboard
%tensorboard --logdir . --bind_all

In [ ]:
#@title Training
dataset = "/path/to/audio/dataset" #@param {type:"string"}
name = "name_of_run" #@param {type:"string"}
save_dir = "/path/to/save/checkpoints" #@param {type:"string"}
architecture = "v2" #@param ["v3", "v2", "v1", "discrete", "onnx", "raspberry" ]
regularization = "wasserstein" #@param ["default", "wasserstein", "spherical"]
channels = 1 #@param {type:"integer"}
batch_size = 2 #@param {type:"integer"}
val_every = 1000 #@param {type:"integer"}
save_every = 500 #@param {type:"integer"}

%cd /content/
!mkdir dataset
%cd $save_dir
preprocessed_dataset = "/content/dataset"

# !/content/miniconda/bin/rave train --helpfull
if regularization == "default":
  !/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --channels $channels
  !/content/miniconda/bin/rave train --config $architecture --db_path $preprocessed_dataset --name $name --val_every $val_every --channels $channels --save_every $save_every --batch $batch_size
else:
  !/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --channels $channels
  !/content/miniconda/bin/rave train --config $architecture --config $regularization --db_path $preprocessed_dataset --name $name --val_every $val_every --channels $channels --batch $batch_size --save_every $save_every

In [ ]:
#@title Resume Training

#@title Training
dataset = "/path/to/audio/dataset" #@param {type:"string"}
name = "run_name" #@param {type:"string"}
save_dir = "/path/to/save/checkpoints" #@param {type:"string"}
architecture = "v2" #@param ["v3", "v2", "v1", "discrete", "onnx", "raspberry" ]
regularization = "wasserstein" #@param ["default", "wasserstein", "spherical"]
resume = "/path/to/rave/checkpoint.ckpt" #@param {type:"string"}
channels = 1 #@param {type:"integer"}
config = "/path/to/rave/config.gin" #@param {type:"string"}
batch_size = 2 #@param {type:"integer"}
val_every = 1000 #@param {type:"integer"}
save_every = 500 #@param {type:"integer"}

%cd /content/
!mkdir dataset
%cd $save_dir
preprocessed_dataset = "/content/dataset"

# !/content/miniconda/bin/rave train --helpfull
if config:
  !/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --channels $channels
  !/content/miniconda/bin/rave train --config $config --db_path $preprocessed_dataset --name $name --ckpt $resume --val_every $val_every --channels $channels --batch $batch_size --save_every $save_every
elif regularization == "default":
  !/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --channels $channels
  !/content/miniconda/bin/rave train --config $architecture --db_path $preprocessed_dataset --name $name --ckpt $resume --val_every $val_every --channels $channels --batch $batch_size --save_every $save_every
else:
  !/content/miniconda/bin/rave preprocess --input_path $dataset --output_path $preprocessed_dataset --channels $channels
  !/content/miniconda/bin/rave train --config $architecture --config $regularization --db_path $preprocessed_dataset --name $name --ckpt $resume --val_every $val_every --channels $channels --batch $batch_size --save_every $save_every

In [ ]:
#@title Export as torchscript (.ts)

model_dir = "/path/to/trained/rave/run" #@param {type:"string"}


!/content/miniconda/bin/rave export --run $model_dir --streaming --fidelity 0.999